In [1]:
from pathlib import Path
import os

In [2]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.readers.file import PDFReader, PyMuPDFReader
# from llama_index.storage.docstore.redis import RedisDocumentStore
# from llama_index.storage.index_store.redis import RedisIndexStore
# from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.core.node_parser import SentenceSplitter

# from redis import Redis
# from redisvl.schema import IndexSchema

In [4]:
# REDIS_HOST = os.getenv("REDIS_HOST", "127.0.0.1")
# REDIS_PORT = os.getenv("REDIS_PORT", 6379)

In [5]:
from IPython.core.display import HTML, Pretty, Markdown

In [6]:
endpoint = "http://phoenix:6006/v1/traces"  # Phoenix receiver address

tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

In [7]:
file_details = {
    "2Q24 Earnings Release_Final.pdf": "2nd quarter 2024 earnings release Final of PNC Bank with detailed financial information for first two quarters of year 2024.",
    "2Q24 Financial Supplement_Final.pdf": "Supplemental data for 2nd quarter 2024 earnings release Final of PNC Bank with financial tables.",
    "Board of Directors  PNC.pdf": "PNC Bank Board of Directors member information.",
    "PNC 2023 10-K.pdf": "PNC Bank Form 10-K report for year 2023.",
    "PNC 2023 Annual Report.pdf": "PNC Bank detailed Annual Report for year 2023.",
    "pnc_privacy_notice.pdf": "Information on what kind of customer personal information PNC Bank share and not.",
}
def get_meta(filename):
    return {
        "file_path": filename,
        "file_details": file_details.get(
            Path(filename).name, ""
        ),
    }

In [7]:
# custom_schema = IndexSchema.from_dict(
#     {
#         # customize basic index specs
#         "index": {
#             "name": "user-index",
#             "prefix": "index",
#             "key_separator": ":",
#         },
#         # customize fields that are indexed
#         "fields": [
#             # required fields for llamaindex
#             {"type": "tag", "name": "id"},
#             {"type": "tag", "name": "doc_id"},
#             {"type": "text", "name": "text"},
#             # custom metadata fields
#             {"type": "numeric", "name": "updated_at"},
#             {"type": "tag", "name": "file_name"},
#             # custom vector field definition for cohere embeddings
#             {
#                 "type": "vector",
#                 "name": "vector",
#                 "attrs": {
#                     "dims": 768,
#                     "algorithm": "hnsw",
#                     "distance_metric": "cosine",
#                 },
#             },
#         ],
#     }
# )

In [8]:
# storage_context = StorageContext.from_defaults(
#     docstore=RedisDocumentStore.from_host_and_port(
#         host=REDIS_HOST, port=REDIS_PORT, namespace="llama_index"
#     ),
#     index_store=RedisIndexStore.from_host_and_port(
#         host=REDIS_HOST, port=REDIS_PORT, namespace="llama_index"
#     ),
#     vector_store=RedisVectorStore(redis_client=Redis.from_url(f"redis://{REDIS_HOST}:{REDIS_PORT}"), 
#                                   overwrite=True,
#                                   schema=custom_schema
#                                   )
# )

In [9]:
# PDF Reader with `SimpleDirectoryReader`
parser = PyMuPDFReader()
file_extractor = {".pdf": parser}

In [10]:
documents = SimpleDirectoryReader(
    "./pnc", file_metadata=get_meta, file_extractor=file_extractor
).load_data()

In [21]:
documents[0]

Document(id_='6e5aaf33-91e9-4d2e-9f34-3e5e456bd1a8', embedding=None, metadata={'file_path': '/home/docqa/app/pnc/2Q24 Earnings Release_Final.pdf', 'file_details': '2nd quarter 2024 earnings release Final of PNC Bank with detailed financial information for first two quarters of year 2024.', 'total_pages': 18, 'source': '1'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='PNC Reports Second Quarter 2024 Net Income of $1.5 Billion, $3.39 Diluted EPS\n   Generated positive operating leverage; grew NII and NIM; maintained 2.5% SCB requirement\nIncreased quarterly common stock dividend 5 cents to $1.60 per share on July 2, 2024\n PITTSBURGH, July 16, 2024 – The PNC Financial Services Group, Inc. (NYSE: PNC) today reported:\nFor the quarter\nIn millions, excep

In [35]:
for i in range(3):
    display(f"{'#' * 20} Page {i} {'#' * 20}")
    display(Pretty(documents[i].text))

'#################### Page 0 ####################'

PNC Reports Second Quarter 2024 Net Income of $1.5 Billion, $3.39 Diluted EPS
   Generated positive operating leverage; grew NII and NIM; maintained 2.5% SCB requirement
Increased quarterly common stock dividend 5 cents to $1.60 per share on July 2, 2024
 PITTSBURGH, July 16, 2024 – The PNC Financial Services Group, Inc. (NYSE: PNC) today reported:
For the quarter
In millions, except per share data and as noted
2Q24
1Q24
2Q23
Second Quarter Highlights
Financial Results
Comparisons reflect 2Q24 vs. 1Q24
Net interest income (NII)
$ 
3,302 
$ 
3,264 
$ 
3,510 
Income Statement
▪Generated positive operating 
leverage; PPNR increased 13%
▪Revenue increased 5%
– NII and NIM increased
▪Noninterest expense increased 1%
▪Gain on Visa share exchange of 
$754 million substantially offset by 
other significant items, resulting in a 
9 cent benefit to EPS
Balance Sheet 
▪Average loans and deposits were 
relatively stable
▪Average securities increased 4%
▪Net loan charge-offs were $262 
million, or 

'#################### Page 1 ####################'

Significant Items
In the second quarter of 2024, PNC participated in the Visa exchange program, allowing PNC to monetize 
50% of its Visa Class B-1 shares and converting its remaining holdings into 1.8 million of Visa Class B-2 shares. The 
exchange resulted in a gain of $754 million. The second quarter of 2024 also included Visa Class B-2 derivative fair 
value adjustments of negative $116 million, primarily related to the extension of anticipated litigation resolution timing, 
and a $120 million expense related to a PNC Foundation contribution. During the quarter, PNC also repositioned the 
investment securities portfolio, selling available-for-sale securities with a market value of $3.8 billion and a weighted 
average yield of approximately 1.5%. The sale of securities resulted in a loss of $497 million. PNC redeployed the 
sale proceeds into available-for-sale securities with a market value of $3.8 billion and a weighted average yield of 
approximately 5.5%. The combined net income

'#################### Page 2 ####################'

▪
Noninterest expense of $3.4 billion increased $23 million, or 1%, and included a $120 million PNC Foundation 
contribution expense in the second quarter of 2024, while the first quarter included a $130 million FDIC special 
assessment expense. 
▪
Provision for credit losses was $235 million in the second quarter, primarily reflecting the impact of portfolio 
activity. The first quarter of 2024 included a provision for credit losses of $155 million.   
▪
The effective tax rate was 18.8% for both the second and first quarter.
Balance Sheet Highlights
Second quarter 2024 compared with first quarter 2024 or June 30, 2024 compared with March 31, 2024
▪
Average loans of $319.9 billion were stable, reflecting average commercial loans of $219.1 billion and average 
consumer loans of $100.8 billion. 
▪
Credit quality performance:
–
Delinquencies of $1.3 billion were stable.
–
Total nonperforming loans of $2.5 billion increased $123 million, or 5%, primarily due to higher commercial 
nonperfor

In [50]:
nodes = list(index.docstore.docs.values())
for i in range(4):
    display(f"{'#' * 20} Page {i} {'#' * 20}")
    display(Pretty(nodes[i].text))

'#################### Page 0 ####################'

PNC Reports Second Quarter 2024 Net Income of $1.5 Billion, $3.39 Diluted EPS
   Generated positive operating leverage; grew NII and NIM; maintained 2.5% SCB requirement
Increased quarterly common stock dividend 5 cents to $1.60 per share on July 2, 2024
 PITTSBURGH, July 16, 2024 – The PNC Financial Services Group, Inc. (NYSE: PNC) today reported:
For the quarter
In millions, except per share data and as noted
2Q24
1Q24
2Q23
Second Quarter Highlights
Financial Results
Comparisons reflect 2Q24 vs. 1Q24
Net interest income (NII)
$ 
3,302 
$ 
3,264 
$ 
3,510 
Income Statement
▪Generated positive operating 
leverage; PPNR increased 13%
▪Revenue increased 5%
– NII and NIM increased
▪Noninterest expense increased 1%
▪Gain on Visa share exchange of 
$754 million substantially offset by 
other significant items, resulting in a 
9 cent benefit to EPS
Balance Sheet 
▪Average loans and deposits were 
relatively stable
▪Average securities increased 4%
▪Net loan charge-offs were $262 
million, or 

'#################### Page 1 ####################'

From Bill Demchak, PNC Chairman and Chief Executive Officer:
“PNC delivered strong results in the second quarter; generating positive operating leverage through revenue 
growth and well controlled expenses while adding customers, and strengthening our capital levels. Importantly, net 
interest income and net interest margin increased, marking the beginning of our growth trajectory towards 
expected record NII in 2025. In June, the Federal Reserve announced the results of the annual stress test and 
PNC’s start-to-trough CET1 ratio depletion was 1.6%, the best in our peer group. And earlier this month, our board 
approved a 5 cent increase to our quarterly common stock dividend.” 
- more -

'#################### Page 2 ####################'

Significant Items
In the second quarter of 2024, PNC participated in the Visa exchange program, allowing PNC to monetize 
50% of its Visa Class B-1 shares and converting its remaining holdings into 1.8 million of Visa Class B-2 shares. The 
exchange resulted in a gain of $754 million. The second quarter of 2024 also included Visa Class B-2 derivative fair 
value adjustments of negative $116 million, primarily related to the extension of anticipated litigation resolution timing, 
and a $120 million expense related to a PNC Foundation contribution. During the quarter, PNC also repositioned the 
investment securities portfolio, selling available-for-sale securities with a market value of $3.8 billion and a weighted 
average yield of approximately 1.5%. The sale of securities resulted in a loss of $497 million. PNC redeployed the 
sale proceeds into available-for-sale securities with a market value of $3.8 billion and a weighted average yield of 
approximately 5.5%. The combined net income

'#################### Page 3 ####################'

▪
Noninterest expense of $3.4 billion increased $23 million, or 1%, and included a $120 million PNC Foundation 
contribution expense in the second quarter of 2024, while the first quarter included a $130 million FDIC special 
assessment expense. 
▪
Provision for credit losses was $235 million in the second quarter, primarily reflecting the impact of portfolio 
activity. The first quarter of 2024 included a provision for credit losses of $155 million.   
▪
The effective tax rate was 18.8% for both the second and first quarter.
Balance Sheet Highlights
Second quarter 2024 compared with first quarter 2024 or June 30, 2024 compared with March 31, 2024
▪
Average loans of $319.9 billion were stable, reflecting average commercial loans of $219.1 billion and average 
consumer loans of $100.8 billion. 
▪
Credit quality performance:
–
Delinquencies of $1.3 billion were stable.
–
Total nonperforming loans of $2.5 billion increased $123 million, or 5%, primarily due to higher commercial 
nonperfor

In [11]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# Groq
Settings.llm = Groq(
    model="llama-3.1-8b-instant"
)

In [38]:
index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True,
    # storage_context=storage_context
)

Parsing nodes:   0%|          | 0/483 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/712 [00:00<?, ?it/s]

In [14]:
query_engine = index.as_query_engine()

In [15]:
response = query_engine.query(
    "Who is PNC Chairman? Provide evidence from documents to support your answer."
)
print(response)

William S. Demchak is the Chairman & Chief Executive Officer of The PNC Financial Services Group. This is supported by the document, which states: "William S. Demchak is chairman and chief executive officer of The PNC Financial Services Group, one of the largest diversified financial services companies in the United States."


In [16]:
response = query_engine.query("Tell me about Bryan Salesky work experiance.")
print(response)

Bryan Salesky has a significant work experience in the technology industry. He co-founded and served as the CEO of Stack AV Co., a developer of autonomous trucking solutions. Prior to this role, he co-founded and served as the CEO of Argo AI, LLC, a self-driving technology platform company. Additionally, he spent over a decade in various roles at leading technology companies, including Google and Carnegie Mellon University's National Robotics Engineering Center (NREC).


In [17]:
response = query_engine.query("What does PNC do with customer personal information?")
print(response)

PNC collects and shares customer personal information for various purposes. This information can include social security numbers, income, account balances, and credit scores. They share this information for their everyday business purposes, such as processing transactions and reporting to credit bureaus. They also share it for marketing purposes, including offering their products and services to customers. Additionally, they share information with affiliates for their everyday business purposes and to market to customers.


In [18]:
response = query_engine.query(
    "What was PNC diluted earnings per common share in 2023? Provide evidence from documents to support your answer."
)
print(response)

According to the provided information, PNC diluted earnings per common share in 2023 was $12.79. This can be found in the "Basic and Diluted Earnings Per Common Share" table in the 2023 Annual Report (Table 77).


In [19]:
response = query_engine.query(
    "How good PNC revenue was in 2023 compared to previous years?"
)
print(response)

PNC generated record revenue of $21.5 billion in 2023, which is a significant improvement compared to previous years.


In [20]:
response = query_engine.query("Can you print a table showing PNC Revenue, Net Income and Total Non-interst Expenses for first two quarters of 2024? Provide evidence from documents to support your answer.")
print(response)

Unfortunately, the provided context information does not contain any data for the first two quarters of 2024. The information available is for the year 2023, specifically the annual reports and Form 10-K documents for that year.

However, I can suggest that you may need to obtain additional information or reports from PNC Financial Services Group, Inc. for the year 2024 to answer your query accurately.


## Unstructured

In [8]:
from unstructured.partition.pdf import partition_pdf

In [22]:
elements_fast = partition_pdf("pnc/2Q24 Earnings Release_Final.pdf", strategy='fast')

In [24]:
elements_fast[0].to_dict()

{'type': 'NarrativeText',
 'element_id': 'fce6578771fbe03b05af31caa11158fb',
 'text': 'PNC Reports Second Quarter 2024 Net Income of $1.5 Billion, $3.39 Diluted EPS Generated positive operating leverage; grew NII and NIM; maintained 2.5% SCB requirement Increased quarterly common stock dividend 5 cents to $1.60 per share on July 2, 2024',
 'metadata': {'coordinates': {'points': ((61.17, 95.77294871999993),
    (61.17, 136.63104465999993),
    (551.2199999999998, 136.63104465999993),
    (551.2199999999998, 95.77294871999993)),
   'system': 'PixelSpace',
   'layout_width': 612.0,
   'layout_height': 792.0},
  'file_directory': 'pnc',
  'filename': '2Q24 Earnings Release_Final.pdf',
  'languages': ['eng'],
  'last_modified': '2024-08-23T14:01:08',
  'page_number': 1,
  'filetype': 'application/pdf'}}

In [25]:
for i in range(15):
    display(f'{"#" *20} Type {elements_fast[i].to_dict()["type"]} {"#" * 20}') 
    display(Pretty(elements_fast[i].to_dict()["text"])) 

'#################### Type NarrativeText ####################'

PNC Reports Second Quarter 2024 Net Income of $1.5 Billion, $3.39 Diluted EPS Generated positive operating leverage; grew NII and NIM; maintained 2.5% SCB requirement Increased quarterly common stock dividend 5 cents to $1.60 per share on July 2, 2024

'#################### Type UncategorizedText ####################'

PITTSBURGH, July 16, 2024 – The PNC Financial Services Group, Inc. (NYSE: PNC) today reported:

'#################### Type Title ####################'

...

$ 3,302 $ 3,264 $ 3,510

'#################### Type UncategorizedText ####################'

2,109

'#################### Type UncategorizedText ####################'

1,881

In [9]:
elements = partition_pdf("pnc/2Q24 Earnings Release_Final.pdf", 
                         strategy='hi_res', 
                         hi_res_model_name='detectron2_onnx',
                        )

model.onnx:   0%|          | 0.00/166M [00:00<?, ?B/s]

In [10]:
elements[0].to_dict()

{'type': 'Image',
 'element_id': '530cc4f5accb1983607c3b49df80dc17',
 'text': 'NEWS RELEASE PNC',
 'metadata': {'coordinates': {'points': ((0.0, 2.777777770764563e-05),
    (0.0, 215.30558333333323),
    (1700.0, 215.30558333333323),
    (1700.0, 2.777777770764563e-05)),
   'system': 'PixelSpace',
   'layout_width': 1700,
   'layout_height': 2200},
  'last_modified': '2024-08-23T14:01:08',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'file_directory': 'pnc',
  'filename': '2Q24 Earnings Release_Final.pdf'}}

In [11]:
for i in range(15):
    display(f'{"#" *20} Type {elements[i].to_dict()["type"]} {"#" * 20}') 
    display(Pretty(elements[i].to_dict()["text"])) 

'#################### Type Image ####################'

NEWS RELEASE PNC

'#################### Type Title ####################'

PNC Reports Second Quarter 2024 Net Income of $1.5 Billion, $3.39 Diluted EPS Generated positive operating leverage; grew NII and NIM; maintained 2.5% SCB requirement Increased quarterly common stock dividend 5 cents to $1.60 per share on July 2, 2024

'#################### Type UncategorizedText ####################'

...

Per Common Share

'#################### Type NarrativeText ####################'

$754 million substantially offset by other significant items, resulting in a 9 cent benefit to EPS

'#################### Type NarrativeText ####################'

Diluted earnings per share (EPS)

In [14]:
for i in range(1500):
    if elements[i].to_dict()["type"] == 'Table':
        display(elements[i].to_dict()["text"])  

'In millions, except per share data and as noted 2Q24 1Q24 2Q23 Second Quarter Highlights Comparisons reflect 2Q24 vs. 1Q24 $ 3,302 $ 3,264 $ 3,510 2,109 1,881 1,783 Income Statement 5,411 3,357 5,145 3,334 5,293 3,372 ▪ Generated positive operating leverage; PPNR increased 13% Pretax, pre-provision earnings (PPNR) (non-GAAP) 2,054 1,811 1,921 ▪ Revenue increased 5% 235 1,477 155 1,344 146 1,500 – NII and NIM increased ▪ Noninterest expense increased 1% ▪ Gain on Visa share exchange of Average diluted common shares outstanding $ 3.39 $ 400 3.10 $ 400 3.36 401 116.70 89.12 113.30 85.70 105.67 77.80 Balance Sheet ▪ Average loans and deposits were relatively stable $ 319.9 $ 320.6 $ 324.5 ▪ Average securities increased 4% Average deposits In billions Accumulated other comprehensive income (loss) (AOCI) In billions Net loan charge-offs 141.3 417.2 (7.4) 262 135.4 420.2 (8.0) 243 141.0 425.7 (9.5) 194 ▪ Net loan charge-offs were $262 million, or 0.33% annualized to average loans ▪ ACL to to

'Average Loans In billions 2Q24 1Q24 2Q23 Change 2Q24 vs 1Q24 Commercial $ 219.1 $ 219.2 $ 223.2 — Consumer 100.8 101.4 101.3 (1) % Total $ 319.9 $ 320.6 $ 324.5 — Change 2Q24 vs 2Q23 (2) % (1) % (1) %'

'Change 2Q24 vs 2Q24 1Q24 2Q23 1Q24 $ 77.5 $ 75.6 $ 65.7 2 % Avg. borrowed funds to avg. liabilities 15 % 15 % 13 %'

'Capital June 30, 2024 March 31, 2024 June 30, 2023 Common shareholders’ equity In billions Accumulated other comprehensive income (loss) In billions $ $ 46.4 (7.4) $ $ 45.1 (8.0) $ $ 42.1 (9.5) Basel III common equity Tier 1 capital ratio * 10.2 % 10.1 % 9.5 % Basel III common equity Tier 1 fully implemented capital ratio (estimated) 10.1 % 10.0 % 9.4 %'

'2Q24 1Q24 2Q23 $ 1,715 $ 1,085 $ 954 1,046 1,121 817 103 97 63 (1,405) (973) (351) Net income excluding noncontrolling interests $ 1,459 $ 1,330 $ 1,483 2Q24 1Q24 2Q23 Change 2Q24 vs 1Q24 Change 2Q24 vs 2Q23 $ 2,709 $ 2,617 $ 2,448 $ 92 $ 261 $ 1,409 $ 764 $ 702 $ 645 $ 707 $ 1,841 $ 1,837 $ 1,904 $ 4 $ $ 27 $ 118 $ (14) $ (91) $ 41 $ 1,715 $ 1,085 $ 954 $ 630 $ 761 $ 96.5 $ 97.2 $ 97.6 $ (0.7) $ $ 249.7 $ 249.0 $ 257.3 $ 0.7 $ $ 138 $ 139 $ 109 $ (1) $ 29'

IndexError: list index out of range